@author: Carson Hanel

Note: These code snippets are derived from Data Science from Scratch First Principles by Joel Grus.
      For right now, I'll be transferring the code from the book, explaining the functions, and creating
      an API that can be utilized in further analysis. While some of these functions may be part of 
      the Python standard library or a package already created, I thought it would be useful to begin
      creating my own data science toolbelt for the future, with self written commentary.
      
Necessary imports/functions:

In [33]:
import math
import random

def normal_cdf(x, mu=0, sigma=1):
  return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
  if mu != 0 or sigma != 1:
    return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
            
  low_z, low_p = -10.0, 0
  hi_z,  hi_p  =  10.0, 1
  while hi_z - low_z > tolerance:
    mid_z = (low_z + hi_z) / 2
    mid_p = normal_cdf(mid_z)
    if mid_p < p:
      low_z, low_p = mid_z, mid_p
    elif mid_p > p:
      hi_z, hi_p = mid_z, mid_p
    else:
      break
                        
  return mid_z

Normal approximation to binomial:
    Coin flipping trial.
    p = 0.5
    Each coin flip is a Bernoulli trial;
    X is a Binomial(n, p) random variable
   
   You can approximate it utilizing:

In [34]:
def normal_approximation_to_binomial(n, p):
  mu = p * n
  sigma = math.sqrt(p * (1 - p) * n)
  return mu, sigma

Explanation:
    Whenever a random variable follows a normal distribution, we can use
    normal_cdf to figure out the probability that its realized value lies
    either within or outside a certain interval.

Noteworthy: The normal CDF is equivalent to the probability that a variable is below a threshold.

In [35]:
normal_probability_below = normal_cdf
  
#It's above the threshold if it's not below the threshold
def normal_probability_above(lo, mu=0, sigma=1):
  return 1 - normal_cdf(lo, mu, sigma)
  
#It's between the threshold if it's less than hi, but not less than lo
def normal_probability_between(lo, hi, mu=0, sigma=1):
  return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

#It's outside if it's not between
def normal_probability_outside(lo, hi, mu=0, sigma=1):
  return 1 - normal_probability_between(lo, hi, mu, sigma)

Furthermore:
    We can also do the reverse, finding the intervals at which likelihood cutoffs exist.
    For example, "if we want to find an interval centered at the mean and containing 60%
    probability, then we find the cutoffs of the upper and lower 20%"
    
    These are the methods we can utilize:

In [36]:
def normal_upper_bound(probability, mu=0, sigma=1):
# returns the z for which P(Z <= z) = probability
  return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
# retuns the z for which P(Z >= z) = probability 
  return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
# returns the symmetric (about the mean) bounds
# that contain the specified probability  
  tail_probability = (1 - probability) / 2
  upper_bound      = normal_lower_bound(tail_probability, mu, sigma)
  lower_bound      = normal_upper_bound(tail_probability, mu, sigma)
  return lower_bound, upper_bound

In order to run the experiment of flipping an unbiased coin 1000 times:

In [37]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

Next:
    We need to make a decision about significance.
    How willing are we to accept an error?  
    In this, let's choose 5% of the time.

In [38]:
normal_two_sided_bounds(0.95, mu_0, sigma_0)

(469.01026640487555, 530.9897335951244)

Continuing:
    Assuming the hypothesis is true, there's at most a 2.5% chance we observe a set of 1000
    trials outside of the tested confidence interval.
    
    Further, we're concerned with the probability that we've made a type 2 error, or rather,
    we've accepted the hypothesis although it is generally false in nature. To find this, we
    calculate the "power" of a test, which is the probability of _not_ making a type 2 error.
    
    Let's check out what happens if p is actually .55 rather than .5

In [39]:
#97.5% bounds based on assumption p is 0.5
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)
      
#actual mu and sigma based on p = .55
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)
      
#a type 2 error means we fail to reject the null hypothesis
#which will happen when x is still in our original interval
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability
print(power)

0.886548001295


Also:
    Imagine that the null hypothesis was instead the the coin was not biased
    towards heads, or rather p <= .5. "In that case we want a one-sided test that rejects the
    null hypothesis when X is much larger than 50 but not when X is smaller than 50. So a 5%
    significance test involves normal_probability_below to find the cutoff below which 95% of
    the probability lies"

In [40]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
  
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
print(power)

0.936379480331


Two-sided bias test:
    To test if the probability is actually .5

In [41]:
def two_sided_p_value(x, mu=0, sigma=1):
  if x>= mu:
  #if x is greater than the mean, the tail is what's greater than x
    return 2 * normal_probability_above(x, mu, sigma)
  else:
  #if x is less than the mean, the tail is what's less than x
    return 2 * normal_probability_below(x, mu, sigma)

If we were to see 530 heads in 1000, we would compute:


In [42]:
two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598857

Tradeskill Note:
    Utilizing 529.5 instead of 530.
    This is called a "continuity correction" which reflects the fact that
    normal_probability_between(529.5, 530.5, mu_0, sigma_0) is a better probability
    of seeing 530 heads than normal_probability between(530, 531, mu_0, sigma_0) is.
    
    Nice.
    
One way to convince yourself that this is a sensible estimate is with a simulation:

In [58]:
extreme_value_count = 0
for _ in range(1000000):
  num_heads = sum(1 if random.random() < 0.5 else 0 for _ in range(1000))
  if num_heads >= 530 or num_heads <= 470:
    extreme_value_count += 1
print extreme_value_count / 1000000

0


Since the p-value is greater than our 5% significance, we don't reject the null. If we instead saw 532 heads, the p-value would be:

In [47]:
print(two_sided_p_value(531.5, mu_0, sigma_0))

0.0463452878378


Which is smaller than the 5% significance, which means we reject the null.
Similarly:

In [48]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below
  
#For a one-sided test, if we saw 525 heads, we would compute:
upper_p_value(524.5, mu_0, sigma_0)

0.06062885772582083

Versus:

In [49]:
upper_p_value(526.5, mu_0, sigma_0)

0.04686839508859242

Before this point, we've only been testing hypothesis probability. When a random variable has distribution, it is wise to utilize a confidence interval to calculate the certainty of the results.

The confidence in a result, if the exact probability is known, according to the central limit theorem should be approximately normal, which means it the confidence can be calculated as such:

math.sqrt(p * (1 - p) / 1000)

But unfortunately, the probability is not known and must be estimated as such:

In [52]:
p_hat = 525. / 1000.
mu    = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
print(sigma)

0.0157916116974


Which, apparently isn't entirely justified.

If you utilize the normal approximation method, you can be 95% confident the output interval contains the true probability:

In [53]:
normal_two_sided_bounds(0.95, mu, sigma)

(0.4940490278129096, 0.5559509721870904)

Which makes sense, right? the 525 out of 1000 was an example for an outcome of an unbiased coin; behold! .5 is within the confidence interval. Science!

To show how the numbers vary, here's an example with 540 heads:

In [54]:
p_hat = 540. / 1000.
mu    = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
print(sigma)
normal_two_sided_bounds(0.95, mu, sigma)

0.0157607106439


(0.5091095927295919, 0.5708904072704082)

Which, although not lying within .5 "fair" range, is an expected outcome. With a 95% confidence interval, 5% of given experiments will be outside of the appropriate bounds.

The next topic that will be covered is P-Hacking:

A procedure that erroneously rejects the null hypothesis only 5% of the time will - by definition - 5% of the time erroneously reject the null hypothesis:

In [57]:
# Flip a fair coin 1000 times, True = heads, False = tails
def run_experiment():
    return [random.random() < 0.5 for _ in range(1000)]

# Using the 5% significance levels
def reject_fairness(experiment):
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejects = len([experiment 
                   for experiment in experiments 
                   if reject_fairness(experiment)])
print num_rejects

46


Essentially, the author is showing you a medium of being able to force the .05 p-value interval to force correlations between variables. The author explains that this technique is a circumvention of common sense and should not be used to "mine features", rather you should apply common sense to your data.

Personally, I believe if you can mine features and explore causation, that's what data science is all about.

The author continues to provide an example of an A/B test